In [226]:
from pandas_datareader import data
import matplotlib.pyplot as plt
import pandas as pd
import requests
import numpy as np
from pandas.io.json import json_normalize
import requests                  
from bs4 import BeautifulSoup    
import re                        
from datetime import datetime    
from time import mktime          
import math

In [227]:
##variables
url = "https://apidojo-yahoo-finance-v1.p.rapidapi.com/market/get-summary"
token = "22990f9b4amsh4df04e987b36e1bp18331cjsn862ed02f8065"
day_begin = "01-01-1987"
day_end = "01-01-2016"
interval = "1d"

In [228]:
#activate the load data function
def get_data_rollin(Stocknames, Shortnames):
    index = 0
    oop = {}
    for names in Shortnames:
        Stockname = Stocknames[index]
        index +=1
        oop[Stockname] = get_info(names)
    return oop
    
    
def get_info(Stock):
    Stock = load_csv_data(Stock, interval, day_begin, day_end)
    #if len(Stock) < 1000:
        #Stock = load_csv_data(Stock, interval, day_begin, day_end)
    return Stock

In [229]:
def _get_crumbs_and_cookies(stock):
    
    url = 'https://finance.yahoo.com/quote/{}/history'.format(stock)
    with requests.session():
        header = {'Connection': 'keep-alive',
                   'Expires': '-1',
                   'Upgrade-Insecure-Requests': '1',
                   'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) \
                   AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36'
                   }
        
        website = requests.get(url, headers=header)
        soup = BeautifulSoup(website.text, 'lxml')
        crumb = re.findall('"CrumbStore":{"crumb":"(.+?)"}', str(soup))

        return (header, crumb[0], website.cookies)

In [230]:
def convert_to_unix(date):
    
    datum = datetime.strptime(date, '%d-%m-%Y')
    
    return int(mktime(datum.timetuple()))


In [231]:
def load_csv_data(stock, interval, day_begin, day_end):
  
    day_begin_unix = convert_to_unix(day_begin)
    day_end_unix = convert_to_unix(day_end)
    
    header, crumb, cookies = _get_crumbs_and_cookies(stock)
    
    with requests.session():
        url = 'https://query1.finance.yahoo.com/v7/finance/download/' \
              '{stock}?period1={day_begin}&period2={day_end}&interval={interval}&events=history&crumb={crumb}' \
              .format(stock=stock, day_begin=day_begin_unix, day_end=day_end_unix, interval=interval, crumb=crumb)
                
        website = requests.get(url, headers=header, cookies=cookies)
        if len(website.text) < 1000:
            print("something with the data of",stock,"went wrong")
        else: print("mhm we've just received a shit ton of data from",stock)
        
        return website.text.split('\n')[:-1]
    
    

In [232]:
def delete_wrong_datasets(List_of_raws, compname):
    index = 0 
    companynames = list()
    for x in list(List_of_raws.values()):
        if len(List_of_raws[compname[index]]) < 500:
            #print(len(List_of_raws[compname[index]]))
            del List_of_raws[compname[index]]
            index += 1
        else:
            companynames.append(compname[index])
            index += 1
    return List_of_raws
            
            

In [233]:
def new_comp_names(List_of_raws, compnames):
    index = 0 
    updated_compnames = list()
    for x in List_of_raws:
        if x in compnames:
            updated_compnames.append(x)
    return updated_compnames
            

In [234]:
def getting_data_seperated(List_of_raws, compname):
    list2 = list()
    compnameindex = -1
    for x in List_of_raws.values():
        compnameindex += 1  
        temp = list()
        for z in x:
            temp.append(z.split(","))
            new_cols = ["Date"] 
        for name in temp[0]:
            if name != "Date":
                name = compname[compnameindex] + name
                new_cols.append(name)
        temp[0] = new_cols
        list2.append(temp)
        
    list2 = creating_dfs(Startprocess[0], list2)
    
    return list2
#df2 = pd.DataFrame(list2)
#df2.transpose()

In [235]:
def creating_dfs(Compnames, List):
    df_all = pd.DataFrame(List[0])
    df_all = df_all.rename(columns=df_all.iloc[0])
    df_all = df_all.drop(0,axis=0)
    index = 1 
    while index < len(List):
        df = pd.DataFrame(List[index]) 
        df = df.rename(columns=df.iloc[0])
        df = df.drop(0,axis=0)
        df_all = pd.merge(df_all,df, on="Date") 
        index += 1

    return df_all

In [236]:
#START THE PROCESS
def start():
    Stockslist = list()
    Stocknames = list()
    End_of_Party = 0
    inputquest = input("Do you want to insert a list or every company on its own \n list/single")
    if inputquest == "list":
        stocklistinsert = input("Paste your Stocklist :)")
        Stocklistinsert = stocklistinsert.split(" ")
        Stocklistraw = Stocklistinsert[1::2]
        Stocklist = list()
        for x in Stocklistraw:
            Stocklist.append(x.upper())
        Stocknamesraw = Stocklistinsert[::2]
        Stocknames = list()
        for x in Stocknamesraw:
            Stocknames.append(x.upper())
            
        return Stocknames, Stocklist
    
    while End_of_Party == 0:
        input_stock = (input("Insert the Name of the Company to add stock, otherwise enter \"done\": "))
        input_stock = input_stock.upper()
        if input_stock == "DONE":
            End_of_Party += 1
        else:
            Stocknames.append(input_stock)
            input_stock = (input("Insert the SHORT NAME (Stockname) to add stock, otherwise enter \"done\": "))
            input_stock = input_stock.upper()
            if input_stock == "DONE":
                End_of_Party += 1
            else:
                Stockslist.append(input_stock)
            continue
        print(Stocknames)
        print(Stockslist)
        return Stocknames, Stockslist

In [237]:
Startprocess = start()
List_of_raws = get_data_rollin(Startprocess[0],Startprocess[1])
List_of_raws = delete_wrong_datasets(List_of_raws, Startprocess[0])
New_compnamelist = new_comp_names(List_of_raws, Startprocess[0])
df_all = getting_data_seperated(List_of_raws, New_compnamelist)
df_all

#Software        //SAP = SAP // APPLE = AAPL // AMnAZON = AMZN // MICROSOFT = MSFT \n",
#Automobil      //BMW = BMW.DE // Volkswagen = VOW3.DE // Peugeot = PEU.F // Mitsubishi = MSBHY\n",
#Banks         //GOLDMAN = GS // JP MORGAN =JPM // DEUTSCHE BANK = DB // HSBC = HSBC\n",
#sonst        //STARBUCKS = SBUX // WALLMART = WMT // ADIDAS = ADS.DE // NIKE = NKE",
#INDEX       //DOW JONES = ^DJI // DAX30 = ^GDAXI // NIKKEI = ^N225 // 
#SAP', 'APPLE', 'AMAZON', 'MICROSOFT', 'BMW', 'VOLKSWAGEN', 'PEUGOT', 'MITSUBISHI', 'GOLDMAN', 'JPMORGAN', 'DEUTSCHE', 'DB', 'HSBC']
#['SAP', 'AAPL', 'AMZN', 'MSFT', 'BMW.DE', 'VOW3.DE', 'PEU.F', 'MSBHY', 'GS', 'JPM', 'BANK', 'HSBC']

#sap sap apple aapl amazon amzn microsoft msft bmw bmw.de volkswagen vow3.de peugot peu.f mitsubishi msbhy goldman gs JPMorgan jpm DEUTSCHE.BANK DB HSBC HSBC

Do you want to insert a list or every company on its own 
 list/single d
Insert the Name of the Company to add stock, otherwise enter "done":  dax
Insert the SHORT NAME (Stockname) to add stock, otherwise enter "done":  ^gdaxi
Insert the Name of the Company to add stock, otherwise enter "done":  done


['DAX']
['^GDAXI']
mhm we've just received a shit ton of data from ^GDAXI


,Date,DAXOpen,DAXHigh,DAXLow,DAXClose,DAXAdj Close,DAXVolume
1,1987-12-30,1005.190002,1005.190002,1005.190002,1005.190002,1005.190002,0
2,1987-12-31,null,null,null,null,null,null
3,1988-01-01,null,null,null,null,null,null
4,1988-01-04,956.489990,956.489990,956.489990,956.489990,956.489990,0
5,1988-01-05,996.099976,996.099976,996.099976,996.099976,996.099976,0
6,1988-01-06,1006.010010,1006.010010,1006.010010,1006.010010,1006.010010,0
7,1988-01-07,1014.469971,1014.469971,1014.469971,1014.469971,1014.469971,0
8,1988-01-08,1026.689941,1026.689941,1026.689941,1026.689941,1026.689941,0
9,1988-01-11,987.520020,987.520020,987.520020,987.520020,987.520020,0
10,1988-01-12,986.890015,986.890015,986.890015,986.890015,986.890015,0


In [275]:
test = df_all

In [276]:
import datetime
test['year'] = pd.DatetimeIndex(test['Date']).year
Indexe = "DAX"
test = test.get(test[Indexe+"Close"] != "null")
test[Indexe+"Close"] = test[Indexe+"Close"].astype(float)
#test.reset_index(inplace=True)
test = test.drop([Indexe+"Open",Indexe+"High",Indexe+"Low",Indexe+"Adj Close",Indexe+"Volume","Date"],axis=1)
test = test.groupby("year").agg(np.mean)

/Users/a1278/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [277]:
test

,DAXClose
year,
1987,1005.190002
1988,1146.976414
1989,1467.301365
1990,1722.412865
1991,1579.507220
1992,1635.772786
1993,1809.457103
1994,2121.065286
1995,2138.497886


In [261]:
test = test.reset_index(inplace=True)
test

AttributeError: 'NoneType' object has no attribute 'reset_index'

In [278]:
test.to_csv("DAX.csv", index=True)